## PostgreSQL
***

Banco de dados muito usados em produção e teste

### Biblioteca assincrona

A biblioteca [asyncpg](https://github.com/MagicStack/asyncpg) é usada para fazer requisições assincronas no banco de dados usando as corotinas do python (async/await)

### Instalação

```
sudo apt-get update
sudo apt-get install postgresql postgresql-contrib pgadmin3
psql -V
```

Instalação da biblioteca do PostgreSQL no python:

```
sudo apt-get install python3-psycopg2
OU
pip install psycopg2-binary
```

### Entrar no postgresql

```
sudo -i -u postgres
psql <nome-do-banco>
\q para sair
```

ou

```
sudo -u postgres psql
```

### Criar um novo usuário

Atualmente, temos apenas a função postgres configurada no banco de dados. Podemos criar novas funções/usuários através da linha de comando com o comando createrole. A flag --interactive irá solicitá-lo os valores necessários.

```
createuser --interactive
Enter name of role to add: victorhad
Shall the new role be a superuser? (y/n) y
```

### Criando um banco de dados

Por padrão, outra suposição que o sistema de autenticação do Postgres faz é que haverá um banco de dados com o mesmo nome da função que está sendo usada para fazer o login, a que a função tem acesso.

Dessa forma, nós criamos um usuário chamado victorhad, essa função irá tentar se conectar ao banco de dados que também é chamado victorhad por padrão. Você pode criar um banco de dados apropriado com o comando createdb.

```
createdb victorhad
dropdb victorhad
```

### Abrir um prompt postgre com o novo usuário

Para conectar-se com autenticação baseada em ident, você precisará de um usuário Linux com o mesmo nome que sua função Postgres e de banco de dados.

Se você não tiver um usuário Linux correspondente disponível, você pode criar um com o comando adduser. Você tem que fazer isso a partir de uma conta com privilégios sudo (não conectado como o usuário postgres):

```
sudo adduser victorhad
```

Uma vez que você tenha uma conta apropriada disponível, você pode tanto alternar e conectar ao banco de dados digitando:

```
sudo -i -u sammy
psql
```

OU

```
sudo -u sammy psql
```

Uma vez conectado, você pode verificar suas informações de conexão atuais digitando:

```
\conninfo
```

Para garantir direito de acesso ao novo usuário

```
GRANT ALL PRIVILEGES ON DATABASE mydb TO myuser;
```

### Modificar a senha do usuário

```
ALTER USER postgres WITH PASSWORD 'nova_senha';
```

### Comandos PSQL

```
\h - Ajuda de comandos SQL
\? - Ajuda de comandos psql
\q - Sai do psql
\dg - Lista regras (roles ou usuários)
\d <tabela> - Esquema da tabela 
\dt - Lista tabelas
\l - Lista databases
SET timezone = 'America/Sao_Paulo';
```

### Backup

```
sudo -u postgres pg_dump postgres > psql.dump
sudo -u postgres dropdb banco
sudo -u postgres createdb banco; psql banco < banco.dump
```

***
### Conexão
***

In [1]:
import psycopg2

In [2]:
# Connection é o objeto que representa o banco de dados
conexao = psycopg2.connect(
    dbname="notebook",
    user="notebook",
    password="notebook",
    host="postgres",
    port=5432
)

In [3]:
# Vamos criar um cursor, ou seja, um iterador que permite navegar e manipular os registros do DB.
cursor = conexao.cursor()

***
### Criando uma tabela
***

In [4]:
# Criar uma tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS cidades (
    id SERIAL PRIMARY KEY,
    cidade TEXT,
    uf VARCHAR(2)
);
""")

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id SERIAL PRIMARY KEY,
    nome VARCHAR NOT NULL,
    cpf VARCHAR(11) NOT NULL,
    email VARCHAR NOT NULL,
    cidade_id INT,
    criado_em DATE NOT NULL,
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);
""")

***
### Inserindo registros
***

In [6]:
cidades = [('Campinas','SP'),
           ('Sao Paulo','SP'),
           ('Rio de Janeiro','RJ')]

In [7]:
cursor.executemany("""
INSERT INTO cidades (cidade, uf) VALUES (%s, %s)
""", cidades)

In [8]:
clientes = [('Regis', '01119239423', 'regis@gmail.com', 1, '2014-06-08'),
            ('Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09'),
            ('Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09'),
            ('Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08')]

In [9]:
cursor.executemany("""
INSERT INTO clientes (nome, cpf, email, cidade_id, criado_em) VALUES (%s, %s, %s, %s, %s)
""", clientes)

In [10]:
# Salvando os dados no BD.
conexao.commit()

***
### Lendo os dados
***

In [11]:
# Registros de clientes
cursor.execute("""
SELECT * FROM clientes;
""")

In [12]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))


In [13]:
# Registros de cidades
cursor.execute("""
SELECT * FROM cidades;
""")

In [14]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Campinas', 'SP')
(2, 'Sao Paulo', 'SP')
(3, 'Rio de Janeiro', 'RJ')


In [15]:
# Registros de clientes e cidades
cursor.execute("""
SELECT * FROM clientes INNER JOIN cidades ON clientes.cidade_id = cidades.id;
""")

In [16]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8), 1, 'Campinas', 'SP')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8), 3, 'Rio de Janeiro', 'RJ')


***
### Alterando os dados
***

In [17]:
id_cliente = 1
novo_email = 'regis_novo@gmail.com'
novo_criado_em = '2014-06-11'

In [18]:
cursor.execute("""
UPDATE clientes
SET email = %s, criado_em = %s
WHERE id = %s
""", (novo_email, novo_criado_em, id_cliente))

In [19]:
conexao.commit()

In [20]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [21]:
for registro in cursor.fetchall():
    print(registro)

(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))
(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))


***
### Deletando um registro
***

In [22]:
id_cliente = 4

In [23]:
cursor.execute("""
DELETE FROM clientes
WHERE id = %s
""", (id_cliente,))

In [24]:
conexao.commit()

In [25]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [26]:
for registro in cursor.fetchall():
    print(registro)

(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))


In [27]:
# Deleta todas as instâncias
cursor.execute("DELETE FROM clientes")
cursor.execute("DELETE FROM cidades")

In [28]:
conexao.commit()

***
### Adicionar uma nova coluna
***

In [29]:
cursor.execute("""
ALTER TABLE clientes
ADD COLUMN bloqueado BOOLEAN;
""")

In [30]:
conexao.commit()

***
### Fechando conexão
***

In [31]:
cursor.execute("DROP TABLE IF EXISTS clientes")
cursor.execute("DROP TABLE IF EXISTS cidades")
conexao.commit()

In [32]:
cursor.close()
conexao.close()